# Run from bootstrap paths

Now we will use the initial trajectories we obtained from bootstrapping to run an MSTIS simulation. This will show both how objects can be regenerated from storage and how regenerated equivalent objects can be used in place of objects that weren't stored.

Tasks covered in this notebook:
* Loading OPS objects from storage
* Ways of assigning initial trajectories to initial samples
* Setting up a path sampling simulation with various move schemes
* Visualizing trajectories while the path sampling is running

In [1]:
%matplotlib inline

In [1]:
import openpathsampling as paths
import numpy as np
import math

# the openpathsampling OpenMM engine
import openpathsampling.engines.openmm as eng

### Loading things from storage

First we'll reload some of the stuff we stored before. Of course, this starts with opening the file.

In [2]:
old_store = paths.AnalysisStorage("ala_mstis_bootstrap.nc")

Warning on use of the timeseries module: If the inherent timescales of the system are long compared to those being analyzed, this statistical inefficiency may be an underestimate.  The estimate presumes the use of many statistically independent samples.  Tests should be performed to assess whether this condition is satisfied.   Be cautious in the interpretation of the data.

********* JAX NOT FOUND *********
 PyMBAR can run faster with JAX  
 But will work fine without it   
Either install with pip or conda:
      pip install pybar[jax]     
               OR                
      conda install pymbar       
*********************************


A lot of information can be recovered from the old storage, and so we don't have the recreate it. However, we did not save our network, so we'll have to create a new one. Since the network creates the ensembles, that means we will have to translate the trajectories from the old ensembles to new ensembles.

In [3]:
print("PathMovers:", len(old_store.pathmovers))
print("Engines:", len(old_store.engines))
print("Samples:", len(old_store.samples))
print("Trajectories:", len(old_store.trajectories))
print("Ensembles:", len(old_store.ensembles))
print("SampleSets:", len(old_store.samplesets))
print("Snapshots:", len(old_store.snapshots))
print("Networks:", len(old_store.networks))

PathMovers: 45
Engines: 3
Samples: 538
Trajectories: 509
Ensembles: 315
SampleSets: 502
Snapshots: 40242
Networks: 1


Loading from storage is very easy. Each store is a list. We take the 0th snapshot as a template (it doesn't actually matter which one) for the next storage we'll create. There's only one engine stored, so we take the only one.

In [4]:
# template = old_store.snapshots[0]
engine = old_store.engines['default']
mstis = old_store.networks[0]
sset = old_store.tag['sampleset']

### initialize engine
if we do not select a platform the fastest possible will be chosen but we explicitly request to use the one in the config file


In [7]:
print(engine.available_platforms())

['Reference', 'CPU', 'CUDA']


In [9]:
platform = 'CUDA' #CUDA
engine.initialize(platform)

print('Engine uses platform `%s`' % engine.platform)

Engine uses platform `CUDA`


In [10]:
sset.sanity_check()

## Running RETIS

Now we run the full calculation. Up to here, we haven't been storing any of our results. This time, we'll start a storage object, and we'll save the network we've created. Then we'll run a new `PathSampling` calculation object.

In [11]:
# logging creates ops_output.log file with details of what the calculation is doing
#import logging.config
#logging.config.fileConfig("logging.conf", disable_existing_loggers=False)
storage.close()

NameError: name 'storage' is not defined

In [12]:
storage = paths.storage.Storage("ala_mstis_production.nc", "w")

In [13]:
storage.snapshots.save(old_store.snapshots[0]);

Before we can sample we still need to set the actual `MoveScheme` which determines the
set of moves to apply to our set of samples and effectively doing the steps in
replica (sampleset) space. We pick the default scheme for mstis and feed it with
the engine to be used.

In [14]:
scheme = paths.DefaultScheme(mstis, engine)

and finally generate the `PathSampler` object to conduct the simulation.

In [15]:
mstis_calc = paths.PathSampling(
    storage=storage,
    sample_set=sset,
    move_scheme=scheme
)
#mstis_calc.save_frequency = 50
mstis_calc.save_frequency = 1000

Now everything is ready: let's run the simulation! The first step takes a little since all
necessary information, i.e. the engines, topologies, initial snapshots, ..., need to be
stored. Then the monte carlo steps will be performed.

In [16]:
#mstis_calc.run(10000)
mstis_calc.run(50000)

Working on Monte Carlo cycle number 50000
Running for 3 hours 14 minutes 44 seconds -  0.23 seconds per step
Estimated time remaining: 0 seconds
DONE! Completed 50000 Monte Carlo cycles.


In [17]:
print(len(storage.steps))

50001


In [18]:
# commented out during development, so we can "run all" and then do more
storage.close()